In [4]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../src/')

In [5]:
import numpy as np
from modelling import ols,ridge
from modelling.sgd import SGD_optimizer
from model_evaluation.param_analysis import evaluate_parameter
from data.create_dataset import *
from visualization.visualize import *
from sklearn.model_selection import  train_test_split
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression,Ridge

In [6]:
X, z = create_dataset('../data/raw/SRTM_data_Norway_1.tif',degree=2)
X_train, X_test, z_train, z_test = train_test_split(X,z, test_size=0.2)
Xscaler = StandardScaler().fit(X_train)
zscaler = StandardScaler().fit(z_train)

#X_train_scl = Xscaler.transform(X_train)
#z_train_scl = zscaler.transform(z_train)
X_train_scl = X_train - np.mean(X_train,axis=0)
z_train_scl = z_train - np.mean(z_train,axis=0)

In [18]:
beta_ols = ols.fit_beta(X_train,z_train)
beta_ols

array([[1279.52003418],
       [ -61.08654707],
       [-112.78098481],
       [  94.09058323],
       [ 313.18299627],
       [ -17.43762423]])

In [58]:
beta_ridge = ridge.fit_beta(X_train,z_train,lmb = 1)
beta_ridge

array([[1276.92166207],
       [ -54.58278288],
       [-107.42024615],
       [  90.98629789],
       [ 306.52911419],
       [ -19.31949627]])

In [73]:
sgd = SGD_optimizer(fit_intercept = True, batch_size = 100, n_epochs = 10000,use_momentum= True, gamma = 0.5,regularization = 'l2', lmb = 0.001, lr=0.01)
beta_class = sgd.fit(X_train[:,1:],z_train)
beta_class

array([[-21.30110718],
       [-80.32553554],
       [ 77.13363727],
       [269.08048   ],
       [-27.01683842]])

In [74]:
sgd.intercept

1263.4228877313233

Ah yes, it's finally working. Seems to be converging better with momentum too. Meaning I can use fewer epochs for same result.

In [86]:
sgdreg = SGDRegressor( random_state=10, fit_intercept = True, max_iter = 100000, penalty='l2',alpha=0.001, eta0 = 1,tol=None)
sgdreg = sgdreg.fit(X_train[:,1:],z_train.ravel())
print(sgdreg.coef_.T)
print(sgdreg.intercept_.T)

[-21.48849416 -80.63323509  77.63233307 269.03041141 -27.09965576]
[1265.06484155]


In [81]:
ridge_sk = Ridge(alpha=0.001).fit(X_train[:,1:],z_train.ravel())
print(ridge_sk.coef_)
print(ridge_sk.intercept_)

[ -61.0797209  -112.77535403   94.08727317  313.17612702  -17.43966092]
1279.5173142563249


In [2]:
import numpy as np
np.asarray([1,2,3]).shape

(3,)

In [9]:
lmbs = np.logspace(-4, 2, 10)
sgd = SGD_optimizer(fit_intercept = False, batch_size = 100, n_epochs = 1000,use_momentum= True, gamma = 0.5,regularization = 'l2', lmb = 0.001, lr=0.01)
mse_train, mse_test, r2_train,r2_test = evaluate_parameter(X_train,z_train, X_test, z_test, 'lambda', lmbs, sgd)

array([1014.47635626, 1014.42309565, 1013.7581943 , 1014.47669309,
       1016.20174727, 1013.80269602, 1013.72552514, 1012.36998676,
       1013.58170006, 1013.06178768])